In [4]:
import openai
import pandas as pd
import tiktoken
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader

In [5]:
# APIkey file
with open('/home/andres/scrappersample/chatbot/filekey.txt','r') as keyfile:
    apiKey=keyfile.read()
apiKey=apiKey.replace('\n','')

In [6]:
openai.api_key=apiKey

In [7]:
# se instancian los embedding y el modelo a utilizar
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

llm_name='gpt-3.5-turbo'
llm = ChatOpenAI(model_name=llm_name, temperature=0,openai_api_key=openai.api_key)

/home/andres/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/andres/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
# se lee el archivo pdf y se procede a crear los splits del documento
pdfdocs=[]

loader=PyPDFLoader(f"beginning_birders_guide_5_Spanish.pdf")
pages = loader.load_and_split()
pdfdocs.extend(pages)

In [3]:
pdfdocs

[Document(page_content='Pregunta a un B iólogo\nGuía de Observación de Aves para Prin cipiantes\n  David Pe arson • M ichae l Moore • CJ Kazil ek\n      Traducido por Jul iana Sol er\nÍndice\nEquipamiento Esencial\n• Guías de Campo\n• Binoculares\n• Vestimenta \nHabilidades para la Identificación de las A ves \n• Pistas de Campo para la Identificación \n• Observ ación S ocial de los P ájaros\nGuía W eb de los Pájaros del Suroeste \n• Perspectiv a general\n• Visión rápida\n• Tarjetas de datos\nEl conteni do de es te paquete puede s er copiado en s u total idad o parc ialmente. Al  utilizar\npartes  de es te paquete, el  crédito y la referenc ia de Pregunta a un Bi ólogo s iempre deben\nconservarse. \nGuía de Observación de Aves par Principiant es   |  http://askabioloigst .asu.edu |  Pregunta a un Biólogo', metadata={'source': 'beginning_birders_guide_5_Spanish.pdf', 'page': 0}),
 Document(page_content='Página - 2\n¿Estoy mirando a un Cac tus\nWren o a un Si nsonte?\n¿Todos  los colibrí

In [8]:
# se crea la vector data store
persist_directory='birders/chroma'
vectordb = Chroma.from_documents(
    documents=pdfdocs,
    embedding=embedding,
    persist_directory=persist_directory
)


In [9]:
vectordb.persist()

In [10]:
# se hace una prueba de siilitud entre una palabra clave y la base de datos vectorial
question='binoculares'
docs = vectordb.similarity_search(question,k=3)

In [11]:
docs

[Document(page_content='Página - 5\nVisualiz ación de un solo círculo - \nLos cilindros de los objetivos están articulados para que puedas\najustar tus binoculares a la anchura de tus ojos. Mue ve los dos\ncilindros de los objetivos para juntarlos o separarlos hasta que la\nimagen que  sea vea a través de ellos sea un c írculo. . \nAlineación - \nLos dos cilindros de los lentes también tienen que  estar en forma\nperfectamente paralela o alineada por el fabricante. Los binoculares\nbaratos a menudo  cuentan con una  mala alineación o pueden\nfácilmente perder la alineación. Si  los dos cilindros de los objetivos\nestán mal alineados, entonces tendrás una visión do ble. Si están sólo\nligeramente desalineados, tus ojos tendrán dificultades para fusionar\nlas dos imágenes. En o casiones tendrás éxito y podrás ver bien la\nimagen, pe ro el forzar la vista provoca fatiga visual y dolores de\ncabeza. Ant es de comprar los prismáticos, sostenlos firme, mira a\ntravés de ellos a un po ste de 

# Metricas

In [79]:
context=["""La pieza más importante del equipamiento para aprender a
identificar los pájaros es, probablemente, un buen libro de pájaros o
una guía de campo. Lo segundo es un par de binoculares. Si sólo
tienes un pequeño presupuesto, una guía de campo es más asequible
y te colocará en el camino correcto mucho más rápido que teniendo
sólo un par de binoculares. Muy pronto, sin embargo, querrás
invertir en un par de binoculares, ya que la observación de pájaros a
simple vista puede volverse rápidamente frustrante.""",
"""
Las Guías de campo contienen las descripciones de los pájaros que
te permiten identificar el ave que estás mirando. Tendrás que elegir
entre una guía que utilice fotografías y una que utilice pinturas de
colores. Aunque muchos principiantes se sienten más atraídos por las
guías con fotografías, unas pinturas exactas de los pájaros suelen ser
más deseables porque un buen artista puede integrar todas las
variaciones de iluminación y postura y mostrar cómo luce el pájaro
típicamente. Tendrás que elegir una guía que cubra la región en que
deseas observar a los pájaros. Algunas guías locales incluyen sólo las
especies más comunes. Este enfoque hace que el proceso sea más
fácil y a menudo es una buena manera de empezar. Sin embargo,
estas pequeñas guías pueden llegar a ser frustrantes al utilizarlas
cuando las aves que estés tratando de identificar no sean las más
comunes. Otras guías incluyen todas las especies, raras y comunes.
Pueden ser muy confusas para un principiante, pero prácticamente
todas las especies que observes estarán en la guía.

""",
"""Una vez que tengas una guía, tienes que encontrar al pájaro que estés
observando en la guía. No tendría ningún sentido ordenar la guía por
orden alfabético, ya que no conoces el nombre del pájaro. En
cambio, la mayoría de las guías se organizan por grupos de aves
relacionadas evolutivamente, nombradas familias, que se colocan
juntas.
Oye, ¡los pájaros en el libro
no están en orden
alfabético! ¿Qué pasa?
Por ejemplo, la familia del somorgujo, la familia de la curruca o la
familia del pato comparten estructuras de formas y comportamientos
similares, ya que están estrechamente relacionadas. Las familias se
ordenan en la guía en función de su historia evolutiva, desde las más
antiguas evolutivamente, como las de los somorgujos, hasta las más
recientes evolutivamente, como las de los gorriones y los mirlos.
Esto se denomina orden filogenético.
Es muy útil aprender algo sobre las diferentes características de las
familias y el orden en que se colocan en el libro antes de ir al campo.
Familiarízate con lo que hace peculiar a los miembros de cada
familia. La mayoría de las guías de campo tendrán una lista de las
familias, o unas páginas codificadas por colores para ayudarte a
encontrar a las familias de aves que estés buscando. Dentro de una
familia, las especies de aves que comparten aspectos aún más
similares se colocan juntas, en un grupo más pequeño denominado
género (géneros pleurales).
Esto significa que las 700 o más especies de una guía de campo
sobre pájaros de América del Norte están agrupadas en unos 300
géneros, dentro de 80 familias. Las guías más locales, por supuesto,
tienen menos opciones.""",
"""Los binoculares aumentarán en gran medida tu disfrute durante la
observación de los pájaros y facilitarán la identificación, al brindarte
una imagen en primer plano de las aves. Sin embargo, hay cientos de
variaciones de prismáticos disponibles en la actualidad y no todos
son adecuados para la observación de las aves. Para un principiante,
las opciones se pueden reducir con relativa facilidad. Estos son los
elementos simples a tener en cuenta:
Potencia de magnificación -
Todos los binoculares vienen con dos números separados por una
"x", como 8 x 30 o 10 x 50. El primer número es la potencia de
magnificación de los prismáticos. Un ave u otro objeto aparecerán 8
ó 10 veces más cerca, dependiendo de la potencia de magnificación.
Las potencias más altas dan vistas más cercanas de las aves, pero son
difíciles de mantener estables y te muestran un campo de visión más
pequeño, por lo que resulta más difícil encontrar el ave. La mayoría
de los principiantes obtendrían mejores resultados con un par de
prismáticos de 7 u 8 de potencia.""",
"""El segundo número, como 35 ó 50 después de la "x," es una medida
de la amplitud de cada lente en milímetros. Un objetivo más amplio
recoge más luz, pero también hace que un binocular sea más grande,
y más pesado. Muchos principiantes se sienten atraídos por los
prismáticos compactos como los de 8 x 20, ya que son muy
pequeños. Sin embargo, estos prismáticos no reúnen suficiente luz.
Recomendamos los prismáticos con una lente de 35-42 mm. Los
binoculares comúnmente disponibles son de 7 x 35 y 8 x 42, y estas
son muy buenas opciones. Para calcular el tamaño de la pupila de
salida, se divide el tamaño de la lente del objetivo por la potencia de
magnificación de los prismáticos."""]

In [80]:
context

['La pieza más importante del equipamiento para aprender a\nidentificar los pájaros es, probablemente, un buen libro de pájaros o\nuna guía de campo. Lo segundo es un par de binoculares. Si sólo\ntienes un pequeño presupuesto, una guía de campo es más asequible\ny te colocará en el camino correcto mucho más rápido que teniendo\nsólo un par de binoculares. Muy pronto, sin embargo, querrás\ninvertir en un par de binoculares, ya que la observación de pájaros a\nsimple vista puede volverse rápidamente frustrante.',
 '\nLas Guías de campo contienen las descripciones de los pájaros que\nte permiten identificar el ave que estás mirando. Tendrás que elegir\nentre una guía que utilice fotografías y una que utilice pinturas de\ncolores. Aunque muchos principiantes se sienten más atraídos por las\nguías con fotografías, unas pinturas exactas de los pájaros suelen ser\nmás deseables porque un buen artista puede integrar todas las\nvariaciones de iluminación y postura y mostrar cómo luce el pájaro\

In [41]:
# usando a Claude como aplicación Rag de referencia
claude_ground_truth = {
    "preguntas_respuestas": [
        {
            "pregunta": "¿Cuáles son las dos piezas más importantes del equipamiento para aprender a identificar pájaros?",
            "respuesta": "Una buena guía de campo y un par de binoculares."
            
        },
        {
            "pregunta": "¿Por qué las pinturas de aves en las guías de campo suelen ser más deseables que las fotografías?",
            "respuesta": "Porque un buen artista puede integrar todas las variaciones de iluminación y postura, mostrando cómo luce el pájaro típicamente."
        },
        {
            "pregunta": "¿Cómo están organizadas la mayoría de las guías de campo de aves?",
            "respuesta": "Por grupos de aves relacionadas evolutivamente, llamadas familias, que se colocan juntas en un orden filogenético."
        },
        {
            "pregunta": "¿Qué potencia de magnificación se recomienda para los binoculares de un principiante en la observación de aves?",
            "respuesta": "Se recomienda una potencia de 7 u 8."
        },
        {
            "pregunta": "¿Cómo se calcula el tamaño de la pupila de salida en los binoculares?",
            "respuesta": "Se divide el tamaño de la lente del objetivo por la potencia de magnificación de los prismáticos."
        }
        
    ]
}

In [42]:
# respuestas de nuestra aplicación Rag
custom_rag_set = {
    "preguntas_respuestas": [
        {
            "pregunta": "¿Cuáles son las dos piezas más importantes del equipamiento para aprender a identificar pájaros?",
            "respuesta": "Las dos piezas más importantes del equipamiento para aprender a identificar pájaros son un buen libro de pájaros o una guía de campo, y un par de binoculares."
        },
        {
            "pregunta": "¿Por qué las pinturas de aves en las guías de campo suelen ser más deseables que las fotografías?",
            "respuesta": "Las pinturas de aves en las guías de campo suelen ser más deseables que las fotografías porque un buen artista puede integrar todas las variaciones de iluminación y postura, mostrando cómo luce el pájaro típicamente. Esto permite una representación más precisa y detallada de las características clave de cada especie, lo que facilita la identificación de las aves, especialmente para principiantes"
        },
        {
            "pregunta": "¿Cómo están organizadas la mayoría de las guías de campo de aves?",
            "respuesta": "La mayoría de las guías de campo de aves se organizan por grupos de aves relacionadas evolutivamente, llamadas familias. Estas familias se colocan juntas en la guía en función de su historia evolutiva, desde las más antiguas hasta las más recientes evolutivamente. Dentro de una familia, las especies de aves que comparten aspectos aún más similares se colocan juntas en un grupo más pequeño denominado género."
        },
        {
            "pregunta": "¿Qué potencia de magnificación se recomienda para los binoculares de un principiante en la observación de aves?",
            "respuesta": "Se recomienda que los principiantes en la observación de aves utilicen binoculares con una potencia de magnificación de 7 u 8."
        },
        {
            "pregunta": "¿Cómo se calcula el tamaño de la pupila de salida en los binoculares?",
            "respuesta": " Para calcular el tamaño de la pupila de salida en los binoculares, se divide el tamaño de la lente del objetivo por la potencia de magnificación de los prismáticos. Por ejemplo, si tienes unos binoculares con una lente de objetivo de 35 mm y una potencia de magnificación de 7x, la pupila de salida sería de aproximadamente 5 mm (35 mm / 7x = 5 mm)."
        },
        
    ]
}

In [43]:
claude_df=pd.DataFrame()
custom_rag_df=pd.DataFrame()

In [44]:
for i,pair in enumerate(claude_ground_truth['preguntas_respuestas']):
    claude_df.loc[i,'Pregunta']=pair['pregunta']
    claude_df.loc[i,'Claude']=pair['respuesta']
for i,pair in enumerate(custom_rag_set['preguntas_respuestas']):
    custom_rag_df.loc[i,'Pregunta']=pair['pregunta']
    custom_rag_df.loc[i,'Custom_RAG']=pair['respuesta']


In [45]:
comparative_df=custom_rag_df.merge(claude_df,how='inner',on='Pregunta')

In [81]:
comparative_df['Contexto']=context

In [82]:
comparative_df.to_csv('comparative.csv',index=False)

In [83]:
comparative_df

,Pregunta,Custom_RAG,Claude,Contexto,contextual_precision,contextual_recall,contextual_relevancy
0,¿Cuáles son las dos piezas más importantes del...,Las dos piezas más importantes del equipamient...,Una buena guía de campo y un par de binoculares.,La pieza más importante del equipamiento para ...,0.0,1.0,1.0
1,¿Por qué las pinturas de aves en las guías de ...,Las pinturas de aves en las guías de campo sue...,Porque un buen artista puede integrar todas la...,\nLas Guías de campo contienen las descripcion...,1.0,1.0,1.0
2,¿Cómo están organizadas la mayoría de las guía...,La mayoría de las guías de campo de aves se or...,Por grupos de aves relacionadas evolutivamente...,"Una vez que tengas una guía, tienes que encont...",0.0,1.0,1.0
3,¿Qué potencia de magnificación se recomienda p...,Se recomienda que los principiantes en la obse...,Se recomienda una potencia de 7 u 8.,Los binoculares aumentarán en gran medida tu d...,0.0,0.0,0.0
4,¿Cómo se calcula el tamaño de la pupila de sal...,Para calcular el tamaño de la pupila de salid...,Se divide el tamaño de la lente del objetivo p...,"El segundo número, como 35 ó 50 después de la ...",0.0,1.0,1.0


In [57]:
os.environ["OPENAI_API_KEY"]=apiKey

In [70]:
from deepeval import evaluate
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRecallMetric
from deepeval.metrics import ContextualRelevancyMetric

In [72]:

def evaluate_answer(question,answ1,answ2,context):
    actual_output =answ1

    expected_output = answ2


    retrieval_context = [context]
    
    model_name="gpt-3.5-turbo"

    conte_prec = ContextualPrecisionMetric(
        threshold=0.7,
        model=model_name,
        #include_reason=True
    )
    
    conte_recall = ContextualRecallMetric(
        threshold=0.7,
        model=model_name,
        #include_reason=True
    )
    
    conte_relev = ContextualRelevancyMetric(
        threshold=0.7,
        model=model_name,
        #include_reason=True
    )
    
    
    test_case = LLMTestCase(
        input=question,
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=retrieval_context
    )
 

    conte_prec.measure(test_case)
    conte_recall.measure(test_case)
    conte_relev.measure(test_case)
    
    
    return conte_prec.score,conte_recall.score,conte_relev.score
   

   

In [73]:
cont_prec,cont_recall,conte_relev=evaluate_answer(question,answ1,answ2,context)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [87]:

for i in comparative_df.index:
    context=comparative_df.loc[i,'Contexto']
    answ1=comparative_df.loc[i,'Custom_RAG']
    answ2=comparative_df.loc[i,'Claude']
    question=comparative_df.loc[i,'Pregunta']
    cont_prec,cont_recall,conte_relev=evaluate_answer(question,answ1,answ2,context)
    comparative_df.loc[i,'contextual_precision']=cont_prec
    comparative_df.loc[i,'contextual_recall']=cont_recall
    comparative_df.loc[i,'contextual_relevancy']=conte_relev
    

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [88]:
comparative_df

,Pregunta,Custom_RAG,Claude,Contexto,contextual_precision,contextual_recall,contextual_relevancy
0,¿Cuáles son las dos piezas más importantes del...,Las dos piezas más importantes del equipamient...,Una buena guía de campo y un par de binoculares.,La pieza más importante del equipamiento para ...,1.0,1.0,1.0
1,¿Por qué las pinturas de aves en las guías de ...,Las pinturas de aves en las guías de campo sue...,Porque un buen artista puede integrar todas la...,\nLas Guías de campo contienen las descripcion...,0.0,1.0,0.0
2,¿Cómo están organizadas la mayoría de las guía...,La mayoría de las guías de campo de aves se or...,Por grupos de aves relacionadas evolutivamente...,"Una vez que tengas una guía, tienes que encont...",1.0,1.0,1.0
3,¿Qué potencia de magnificación se recomienda p...,Se recomienda que los principiantes en la obse...,Se recomienda una potencia de 7 u 8.,Los binoculares aumentarán en gran medida tu d...,1.0,1.0,1.0
4,¿Cómo se calcula el tamaño de la pupila de sal...,Para calcular el tamaño de la pupila de salid...,Se divide el tamaño de la lente del objetivo p...,"El segundo número, como 35 ó 50 después de la ...",1.0,1.0,1.0


In [86]:
comparative_df.to_csv('comparative.csv',index=False)